In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            soup = BeautifulSoup(resp.text, 'lxml')
                            html = etree.HTML(str(soup))
                
                            title = html.xpath('//h1[@itemprop="name"]/text()')[0].strip()
                            if html.xpath('//li[@class="global-views-breadcrumb-item"]/a[@data-touchpoint="home"]/text()')[0].strip() == 'Home' and html.xpath('//li[@class="global-views-breadcrumb-item-active"]/text()')[0].strip() == title:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_src = html.xpath('//ul[@class="bxslider"]/li[@class="product-details-image-gallery-container"]/noscript/img[@itemprop="image"]/@src') + html.xpath('//div[@class="product-details-image-gallery"]/div[@class="product-details-image-gallery-detailed-image"]/noscript/img[@class="center-block"]/@src')
                list_src = sorted(list(set(list_src)), key=list_src.index)

                # = = = = = = = = = = = = = = =

                dict_src = {}
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = src.strip()

                # = = = = = = = = = = = = = = =

                sku = html.xpath('//span[@itemprop="sku"]/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                dict_description, list_tr = {}, html.xpath('//form[@id="product-details-full-form"]/table[@class="specs-table table table-striped"]/tbody/tr')
                for i, tr in zip(range(len(list_tr)), list_tr):
                    dict_description[str(i)] = {tr.xpath('./td[1]/text()')[0].strip(): tr.xpath('./td[2]/text()')[0].strip()}
    
                # = = = = = = = = = = = = = = =

                list_oem = [text.strip() for text in html.xpath('//form[@id="product-details-full-form"]/div[@data-view="Additional.Detail.Fields"]/div/table[@class="specs-table table table-striped"]/tbody/tr/td[1]/text()')]
                list_oem = sorted(list(set(list_oem)), key=list_oem.index)
                
                oem = ';'.join(list_oem)

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'SKU': sku,
                                         'Title': title,
                                         'OEM': oem,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Description': json.dumps(dict_description)}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：12

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 4.https://www.auveco.com/metric-fast-asst-in-40-comp-bin-1040
[剩余数量：7] - [当前时间：15:50:08]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://www.auveco.com/metric-fastener-assortment-in-two-40-compartment-bins-1042
[剩余数量：6] - [当前时间：15:50:08]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://www.auveco.com/metric-fastener-asst-reg-pitch-w-o-bin-1041
[剩余数量：5] - [当前时间：15:50:09]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://www.auveco.com/metric-fastener-assortment-w-o-bin-1043
[剩余数量：4] - [当前时间：15:50:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://www.auveco.com/uss-fastener-asst-wo-bin-1031
[剩余数量：3] - [当前时间：15:50:10]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 10.https://www.auveco.com/import-retainer-assortment-in-36-comp-bin-1054
[剩余数量：2] - [当前时间：15:50:13]

[状态：ok，尝试次数：2] - 50.00% - crawler_1 > 9.https://www.auveco.com/import-retainer-assortment-w-o-bin-1055
[剩余数量：1] - [当前时间：15:50:14]

[状态：ok，尝试次数：18] - 50.00% - crawler_1 > 7.https://www.auve

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 2167.04it/s]

Done ~
